In [1]:
import sys
sys.path.append("/media/shadowmotion/0CD113590CD11359/code/demo/smp/GuaranteedDelivery")
# sys.path.append("d:\\code\\demo\\smp\\GuaranteedDelivery")
import warnings
warnings.filterwarnings('ignore', category=ResourceWarning)

In [2]:
from core.models.chatglm2.jina_client import encode
from core.prompt import intent_recognition_prompt
from core.prompt import entity_recognition_prompt
from core.data import JinaEmbeddings

from langchain.vectorstores import Weaviate
from langchain.schema import Document
import weaviate

import json

In [3]:
client = weaviate.Client(
    url="http://localhost:8080",  # Replace with your endpoint
    auth_client_secret=weaviate.AuthApiKey(api_key="shadowmotion-secret-key"))

embedding = JinaEmbeddings("127.0.0.1")

In [4]:
with open("../data/chatglm_llm_fintech_raw_dataset/uuid.json", "r") as f:
    uuid_dict = json.load(f)
def attain_uuid(entities, uuid_dict):
    for k, v in uuid_dict.items():
        fg = True
        for entity in entities:
            if entity not in k:
                fg = False
                break
        if fg:
            print(entities, k)
            return v
    return None


In [5]:
def parse_entity_recognition(response: str):
    parse_list = []
    lines = response.split('\n')
    for line in lines:
        sep = ':' if ':' in lines[-1] else '：'
        if "公司名" in line:
            parse_list.append(line.split(sep)[1])
        if "年份" in line:
            parse_list.append(line.split(sep)[1])
    return parse_list
def parse_intent_recognition(response: str):
    lines = response.split('\n')
    return lines[-1]

In [6]:
questions = []
# with open('D:\\code\\demo\\smp\\GuaranteedDelivery\\data\\chatglm_llm_fintech_raw_dataset\\test_questions.jsonl', 'r', encoding='utf-8') as f:
with open('/media/shadowmotion/0CD113590CD11359/code/demo/smp/GuaranteedDelivery/data/chatglm_llm_fintech_raw_dataset/test_questions.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        questions.append(data['question'])

In [7]:
prompt = entity_recognition_prompt("博云新材在2020年对联营企业和合营企业的投资收益是多少元？")

In [8]:
response = encode(prompt, history=[])

In [9]:
print(response[0].text)
r = parse_entity_recognition(response[0].text)
attain_uuid(r, uuid_dict)

公司名：博云新材
年份：2020年
['博云新材', '2020年'] 2021-03-27__湖南博云新材料股份有限公司__002297__博云新材__2020年__年度报告


218751418353274469826944992019657509108

In [10]:
prompt = intent_recognition_prompt("博云新材在2020年对联营企业和合营企业的投资收益是多少元？")

In [11]:
response = encode(prompt, history=[])

In [12]:
response[0].text

'思考：\n1. 题目中出现了具体公司名称的关键词 "博云新材"。\n2. 由于题目包含具体公司名称的关键词，判断该题目属于检索问题。\n答案: 检索问题'

In [13]:
print(response[0].text)
parse_intent_recognition(response[0].text)

思考：
1. 题目中出现了具体公司名称的关键词 "博云新材"。
2. 由于题目包含具体公司名称的关键词，判断该题目属于检索问题。
答案: 检索问题


'答案: 检索问题'

In [14]:
attain_uuid(["汉宇集团股份有限公司", "2020"], uuid_dict)

['汉宇集团股份有限公司', '2020'] 2020-02-29__汉宇集团股份有限公司__300403__汉宇集团__2019年__年度报告


67652975731946022315610746591998057209

In [19]:
with open("../data/test_temp/pipeline.txt", "w") as f:
    for i, question in enumerate(questions):
        if i >= 100:
            break
        # question = "广东冠豪高新技术股份有限公司2019年的法定代表人是谁？"
        f.write(f"Q[{i}]:{question}\n")
        if "2019年" not in question:
            f.write("未录入，跳过\n")
            continue
        prompt = intent_recognition_prompt(question)
        response = encode(prompt, history=[])
        f.write(f"R:\n{response[0].text}\n\n")
        if "检索问题" not in parse_intent_recognition(response[0].text):
            f.write("开放问题直接作答\n")
            response = encode(question, history=[])
            f.write(f"R:\n{response[0].text}\n\n")
            continue
        prompt = entity_recognition_prompt(question)
        response = encode(prompt, history=[])
        f.write(f"R:\n{response[0].text}\n\n")
        entities = parse_entity_recognition(response[0].text)
        uuid = attain_uuid(entities, uuid_dict)
        f.write(f"R:\n{uuid}\n\n")
        if not uuid:
            f.write("未知公司不予作答\n")
            continue
        index_name = f"LangChain_{uuid}"
        try:
            db = Weaviate(client=client, embedding=embedding,
                index_name=index_name, text_key="text", by_text=False)
            
            for word in entities:
                question = question.replace(word, '')
                
            docs = db.similarity_search(question, k=5)

            for i, e in enumerate(docs):
                f.write(f" = = = = = = = = = = = k[{i}] = = = = = = = = = = =\n")
                f.write(e.page_content)
                f.write("\n")
        except:
            f.write("数据库暂未录入\n")

        

['江化微', '2019年'] 2020-03-12__江阴江化微电子材料股份有限公司__603078__江化微__2019年__年度报告


ERROR  GRPCClient@634985 gRPC error: StatusCode.UNAVAILABLE failed to connect to all addresses  [08/04/23 09:06:27]
       None                                                                                                        
       The ongoing request is terminated as the server is not available or closed already.                         

Exception in thread Thread-369:
Traceback (most recent call last):
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/jina/clients/base/grpc.py", line 142, in _get_results
    async for response in stream_rpc.stream_rpc_with_retry():
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/jina/clients/base/stream_rpc.py", line 51, in stream_rpc_with_retry
    async for resp in stub.Call(
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/grpc/aio/_call.py", line 326, in _fetch_stream_responses
    await self._raise_for_status()
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/grpc/aio/_call.py", line 236, in _raise_for_status
    raise _create_rpc_error(await self.initial_metadata(), await
grpc.aio._call.AioRpcError: <AioRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1

['四方科技', '2019年'] 2020-04-16__四方科技集团股份有限公司__603339__四方科技__2019年__年度报告


ERROR  GRPCClient@634985 gRPC error: StatusCode.UNAVAILABLE failed to connect to all addresses  [08/04/23 09:06:40]
       None                                                                                                        
       The ongoing request is terminated as the server is not available or closed already.                         

...

ERROR  GRPCClient@634985 gRPC error: StatusCode.UNAVAILABLE failed to connect to all addresses  [08/04/23 09:15:47]
       None                                                                                                        
       The ongoing request is terminated as the server is not available or closed already.                         

Exception in thread Thread-483:
Traceback (most recent call last):
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/jina/clients/base/grpc.py", line 142, in _get_results
    async for response in stream_rpc.stream_rpc_with_retry():
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/jina/clients/base/stream_rpc.py", line 51, in stream_rpc_with_retry
    async for resp in stub.Call(
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/grpc/aio/_call.py", line 326, in _fetch_stream_responses
    await self._raise_for_status()
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/grpc/aio/_call.py", line 236, in _raise_for_status
    raise _create_rpc_error(await self.initial_metadata(), await
grpc.aio._call.AioRpcError: <AioRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1

['新兴铸管股份有限公司', '2019年'] 2020-04-24__新兴铸管股份有限公司__000778__新兴铸管__2019年__年度报告


ERROR  GRPCClient@634985 gRPC error: StatusCode.UNAVAILABLE failed to connect to all addresses  [08/04/23 09:16:01]
       None                                                                                                        
       The ongoing request is terminated as the server is not available or closed already.                         

Exception in thread Thread-486:
Traceback (most recent call last):
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/jina/clients/base/grpc.py", line 142, in _get_results
    async for response in stream_rpc.stream_rpc_with_retry():
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/jina/clients/base/stream_rpc.py", line 51, in stream_rpc_with_retry
    async for resp in stub.Call(
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/grpc/aio/_call.py", line 326, in _fetch_stream_responses
    await self._raise_for_status()
  File "/home/shadowmotion/anaconda3/envs/py10-117/lib/python3.10/site-packages/grpc/aio/_call.py", line 236, in _raise_for_status
    raise _create_rpc_error(await self.initial_metadata(), await
grpc.aio._call.AioRpcError: <AioRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1

In [ ]:
with open("../data/test_temp/entity.txt", "w") as f:
    for i, question in enumerate(questions):
        f.write(f"Q[{i}]:{question}\n")
        prompt = entity_recognition_prompt(question)
        response = encode(prompt, history=[])
        f.write(f"R:\n{response[0].text}\n\n")
        if i > 10:
            break

In [ ]:
with open("../data/test_temp/intent.txt", "w") as f:
    for i, question in enumerate(questions):
        f.write(f"Q[{i}]:{question}\n")
        prompt = intent_recognition_prompt(question)
        response = encode(prompt, history=[])
        f.write(f"R:\n{response[0].text}\n\n")
        if i > 30:
            break